## Forecasting Model for the COVID-19 Epidemic

In [1]:
# Import Libraries 
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from fbprophet import Prophet
#np.random.seed(42)

In [2]:
summary = pd.read_csv("covid_19_clean_complete.csv", parse_dates = ['Date']).fillna(0)

In [3]:
# Preparing Data
time_series_data = summary[['Date', 'Confirmed']].groupby('Date', as_index = False).sum()
time_series_data.columns = ['ds', 'y']
time_series_data.ds = pd.to_datetime(time_series_data.ds)

In [4]:
# Train-Test Split
train_range = np.random.rand(len(time_series_data)) < 0.8
train_ts = time_series_data[train_range]
test_ts = time_series_data[~train_range]
test_ts = test_ts.set_index('ds')

In [5]:
# Training Prophet Model
prophet_model = Prophet()
prophet_model.fit(train_ts)

INFO:numexpr.utils:NumExpr defaulting to 4 threads.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [6]:
# Testing Model
future = pd.DataFrame(test_ts.index)
predict = prophet_model.predict(future)
forecast = predict[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
forecast = forecast.set_index('ds')

In [7]:
# Mean Absolute Error
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(test_ts.y.values, predict.yhat.values)
print("Mean Absolute Error:", mae)

Mean Absolute Error: 24208.100621488473


In [8]:
# R2 Score
from sklearn.metrics import r2_score
r2 = r2_score(test_ts.y.values, predict.yhat.values)
print("R2 Score:", r2)

R2 Score: 0.9996744902533652


In [9]:
# Mean Squared Error
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(test_ts.y.values, predict.yhat.values)
print("Mean Squared Error:", mse)

Mean Squared Error: 2514388032.3959646


In [25]:
# Plotting Actual vs Predicted
test_fig = go.Figure() 
test_fig.add_trace(go.Scatter(
                x= test_ts.index,
                y= test_ts.y,
                name = "Actual",
                mode = "lines+markers",
                opacity = 1,
                line=dict(color='blue', width=2, dash='dash')))
test_fig.add_trace(go.Scatter(
                x= forecast.index,
                y= forecast.yhat,
                name= "Prediction",
                mode = 'lines',
                opacity = 0.7,
                line = dict(color='red', width=2)))
test_fig.update_layout(title='Forecast Model on Test Data', xaxis_title="Date", yaxis_title="Number of Confirmed Cases",)

test_fig.show()

In [11]:
# Prediction using Forecasting Model
prophet_model_full = Prophet()
prophet_model_full.fit(time_series_data)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [12]:
future_full = prophet_model_full.make_future_dataframe(periods=150)
forecast_full = prophet_model_full.predict(future_full)
forecast_full = forecast_full.set_index('ds')

In [24]:
# Plotting Forecast
prediction_fig = go.Figure() 
prediction_fig.add_trace(go.Scatter(
                x= time_series_data.ds,
                y= time_series_data.y,
                name = "Actual",
                opacity= 1,
                line=dict(color='blue', width=3, dash='dash')))
prediction_fig.add_trace(go.Scatter(
                x= forecast_full.index,
                y= forecast_full.yhat,
                name = "Prediction",
                mode = "lines",
                opacity= 0.7,
                line=dict(color='red', width=2)))
prediction_fig.update_layout(title='Forecasting Model', xaxis_title="Date", yaxis_title="Number of Confirmed Cases")

prediction_fig.show()

In [27]:
test_fig.write_html('testing model.html')
prediction_fig.write_html('prediction model.html')